In [1]:
def NDCG(ui_matrix, est_matrix, k):
    null=[]
    #If number of not-null in each user is greater than k, the calculation will be failed
    #thus those row should be removed from calculation
    for i in range(len(ui_matrix)):
        a=np.count_nonzero(~np.isnan(ui_matrix[i]))
        if a<k:
            null.append(i)
    ui_matrix=np.delete(ui_matrix, [null], 0)
    est_matrix=np.delete(est_matrix, [null], 0)
    
    
    #NDCG calculation
    dcg_sum=0
    idcg_sum=0
    for i in range(len(ui_matrix)):
        #Exclues all the null value from crresponding user(row) of both matrix
        rui=ui_matrix[i]
        rui_bar=est_matrix[i]
        rui=rui[~np.isnan(rui)]
        rui_bar=rui_bar[~np.isnan(rui_bar)]
        
        #Ranking list of estimated rating(rui_bar)
        temp = rui_bar.argsort()[::-1]
        est_rank = np.empty_like(temp)
        est_rank[temp] = np.arange(len(est_rank))
        est_rank=list(est_rank)
        
        #DCG value of each user (Summed up in one value, dcg_sum)
        dcg_sum+=rui[est_rank.index(0)]
        for j in range(1, k):
            dcg_sum+=(rui[est_rank.index(j)]/math.log2(j+1))
        
        #Ranking list of original rating(rui)
        temp = rui.argsort()[::-1]
        ori_rank = np.empty_like(temp)
        ori_rank[temp] = np.arange(len(ori_rank))
        ori_rank=list(ori_rank)
        
        #IDCG value of each user (Summed up in one value, idcg_sum)
        idcg_sum+=rui[ori_rank.index(0)]
        for j in range(1, k):
            idcg_sum+=(rui[ori_rank.index(j)]/math.log2(j+1))

    dcg=dcg_sum/np.abs(len(ui_matrix))
    idcg=idcg_sum/np.abs(len(ui_matrix))
    print("DCG : %f"%(dcg))
    print("IDCG : %f"%(idcg))
    print("NDCG : %f"%(dcg/idcg))

In [2]:
import pandas as pd
import numpy as np

In [3]:
rating_file = pd.read_csv("../dataset/knnbasic_result.csv")

In [4]:
rating_file.head()

,Unnamed: 0,user,item,r_ui,est,err
0,0,Chad,China King Super Buffet,3.0,3.112563,0.112563
1,1,Chad,Panda Express,3.0,2.717246,0.282754
2,2,Margret,Gap,3.0,3.462684,0.462684
3,3,Shannon,Encore Beach Club,2.0,2.832106,0.832106
4,4,Jennifer,Wright Bar,4.0,3.463067,0.536933


In [5]:
predict_csv = rating_file[["user","item","est"]]

In [6]:
predict_csv.head()

,user,item,est
0,Chad,China King Super Buffet,3.112563
1,Chad,Panda Express,2.717246
2,Margret,Gap,3.462684
3,Shannon,Encore Beach Club,2.832106
4,Jennifer,Wright Bar,3.463067


In [7]:
predict_matrix = predict_csv.reset_index().pivot_table(index="user",columns="item",values="est")

In [8]:
predict_matrix.shape

(380, 21601)

In [9]:
est_matrix = predict_matrix

In [11]:
est_matrix

item,#1 Hawaiian Barbecue,'1871' Berkeley Church,00 Gelato,0109 Dessert & Chocolate,1 For the Road,1 Hawaiian Barbecue,1 OAK,1 Pot,100 Degrees Hot Pot,100 Percent Korean,...,täkō,uBreakiFix,uFly Simulator,unPhogettable,z-teca Gourmet Burritos,zpizza,À la Folie,Épices de Cru,Überrito,é by José Andrés
user,,,,,,,,,,,,,,,,,,,,,
Aaron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adrienne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.876335
Yonnie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yuko,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.138857,NaN,4.107267,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
test = pd.read_csv("../dataset/review_top.csv")
test.head()

,user,store,stars
0,Chad,China King Super Buffet,3.0
1,Chad,Panda Express,3.0
2,Margret,Gap,3.0
3,Shannon,Encore Beach Club,2.0
4,Jennifer,Wright Bar,4.0


In [13]:
real_matrix = test.reset_index().pivot_table(index="user",columns="store",values="stars")
real_matrix.shape

(380, 21601)

In [14]:
ui_matrix=real_matrix

In [16]:
rating = np.load("../dataset/real_matrix.npy")

In [19]:
np.save("../dataset/est_matrix_SVD.npy",est_matrix)

In [20]:
predict = np.load("../dataset/est_matrix_SVD.npy")

In [21]:
ui_matrix = rating
est_matrix = predict

In [25]:
NDCG(ui_matrix, est_matrix, k=10)

DCG : 24.174480
IDCG : 25.198195
NDCG : 0.959373


In [24]:
import math

In [28]:
save_path = './dataset/est_matrix_'
algo = str(24)
hwak = '.npy'
total = save_path + algo + hwak

In [29]:
total

'./dataset/est_matrix_24.npy'